In [1]:
import requests
import pprint
import selenium
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
from bs4.element import Comment
import urllib.request
import sys
import os
from os import path
from datetime import datetime
import csv
def all_pages(base_url, links_only=True):
    response=requests.get(base_url)
    unique_urls={base_url}
    visited_urls=set()
    unables=set()
    while len(unique_urls)>len(visited_urls):
        soup=BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a'):
            try:
                url=link['href']                
                parsed_uri = urlparse(url )
                if parsed_uri.netloc=='':
                    absolute_url=base_url+url    
                elif parsed_uri.netloc==base_domain:
                    absolute_url=url
                else:
                    continue
                clean=re.sub(r'[_+!@#$?\\\s]+$', '', absolute_url).replace('.edu','.edu/').replace('.edu//','.edu/')
                unique_urls.add(clean)               
            except:
                continue
        unvisited_url=(unique_urls-visited_urls-unables).pop()
        visited_urls.add(unvisited_url)
        response=requests.get(unvisited_url)
        if links_only!=True and response.status_code=='200':
            handleOnePage(unvisited_url)
        else:
            unables.add(unvisited_url)  
    return unique_urls
def read_oneurl(url):  
    response=requests.get(url)  
    if response.status_code==200: 
        return get_pagetext(response)
    else: 
        return [[None, None, None, None]]
    
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True 

def get_pagetext(body):
    soup = BeautifulSoup(body.text, 'html.parser')
    texts = soup.findAll(text=True) 
    visible_texts = filter(tag_visible, texts)       
    
    return [ [t.parent.name,   
             t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
             t.nextSibling.name if t.nextSibling!=None else None,
             re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 

def save_tocsv(url,folder=None, filename=None): # tab delimiter only
    url=url.strip()
    content=read_oneurl(url) #in format of (a,a,a,a)
     
    if folder==None:
        folder=os.getcwd()
    if path.isdir(folder)==False:
        print('folder doesnot exist')
        return
    if filename==None:
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
    
    fullname=path.join(folder, filename)
    with open(fullname, 'w', newline='',encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
        
        for lll in content: 
            lll.insert(0, url)
            writer.writerow(lll)  
    return fullname           
for i in range(4):
    a=all_pages('http://'+input('input website: '))
    for x in a:
        save_tocsv(x)

input website: mit.edu
input website: h


ConnectionError: HTTPConnectionPool(host='h', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E96F700F88>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))